In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Імпортуємо одразу все, що нам знадобиться

In [ ]:
pd.__version__

# Об'єднання даних кількох датасетів

### pd.concat

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])

In [ ]:
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])

In [ ]:
df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])

In [ ]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])

In [ ]:
display(df1, df2, df3, df4)

In [ ]:
pd.concat([df1, df2])
# Він поєднує по індексу! Тобто, якщо індекси однакові, то вони будуть в одному рядку (якщо відсутні, то вони будуть просто поряд)
# працює як union в SQL
# Якщо відсутній стовпчик, то він буде просто доданий з NaN
#pd.concat([df1, df2, df3])

In [ ]:
pd.concat([df1, df4]) # увага на індекси!

In [ ]:
pd.concat([df1, df4]).reset_index() 

In [ ]:
pd.concat([df1, df4]).reset_index(drop=True) # видаляємо старі індекси

In [ ]:
pd.concat([df1, df4], ignore_index=True) # аналогічно

In [ ]:
pd.concat([df1, df3], axis=1) 
# поєднуємо по стовпчиках
# зазвичай, для цього краще використовувати merge

In [ ]:
# як перевірити дані перед початком обєднання
pd.concat(
    [df1, df4],
    keys=["clients's data","df4"], # додаємо ключі для розрізнення даних 
    names=["Source", "client_id"], # додаємо імена для індексів
)

### pd.merge

In [ ]:
left = pd.DataFrame(
    {
        "ID": ["1", "2", "3", "5", "9"],
        "Name": ["Alex", "Amy", "Allen", "Alice", "Ayoung"],
        "subject_id": ["apple", "orange", "banana", "strawberry", "raspberry"],
    }
)

right = pd.DataFrame(
    {
        "ID": ["1", "1", "3", "5"],
        "col_a": ["A0", "A1", "A2", "A3"],
        "col_b": ["B0", "B1", "B2", "B3"],
        "subject_id": ["apple", "orange", "banana", "kiwi"],
    }
)

display(left, right)

In [ ]:
pd.merge(left, right)
# вона є аналоічною до join в SQL
# по замовчуванню використовується inner join
# використовується спільний стовпчик subject_id
# використовується спільний стовпчик ID
# документація https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

In [ ]:
pd.merge(left, right, suffixes=("_l", "_r"),
         left_index=True, 
         right_index=True, 
         how =  'left') # вказуємо, по якому стовпчику ми хочемо зєднати

# можна поєднати по індексу (left_index=True, right_index=True) і буде як concat 
# fле в проді так не роблять!

In [ ]:
pd.merge(left, right, on="ID")  # вказуємо, по якому стовпчику ми хочемо зєднати

In [ ]:
pd.merge(
    left, right, on=["ID", "subject_id"]
)  # вказуємо кілька стовпчиків, по яким ми хочемо зєднати 

In [ ]:
pd.merge(left, right, suffixes=("_l", "_r"), how="left", left_on="ID", right_on="ID") 
# вказуємо поля для лівої і правої таблиць окремо
# вказуємо cуфікси для стовпчиків, які збігаються

In [ ]:
# додаємо тип з'єднання
pd.merge(left, right, on=["ID", "subject_id"], how="left")

In [ ]:
# додаємо тип з'єднання
pd.merge(left, right, on=["ID", "subject_id"], how="outer")

In [ ]:
pd.merge(left, right, on=["ID", "subject_id"], how="outer", indicator=True) # додаємо індикатор, який показує, звідки взялися дані
# Трішки бонуса, щоб спростити собі життя в майбутньому

### pd.join

In [ ]:
left.join(right, lsuffix="_l", rsuffix="_r")
# Метод dataFrame, a не функція pandas
# join робить те саме що і merge, але простенький 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

# join - це просто merge по індексу
# merge - це join по індексу або по стовпчику, або по кількох стовпчиках = any key, more flexible

In [ ]:
left.join(right,  how="inner", lsuffix="_l", rsuffix="_r")

# внутрішнє зєднання

# Зведені таблиці

In [ ]:
df = pd.read_csv('survey_results_public.csv')
df.head(5)

#завантажимо вже знайомий датасет з опитування StackOverflow

In [ ]:
# будувати зведені таблиці краще за все поступово: від простих до більш складних операцій
# > 100 MB Excel не відкриє, але pandas може працювати з файлами > 1 GB

# pivot_table(data, 
#             values = None, 
#             index = None, 
#             columns = None, 
#             aggfunc = 'mean', 
#             fill_value = None, 
#             margins = False, 
#             dropna = True, 
#             margins_name = 'All')

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df.pivot_table(index="Industry", aggfunc="count").head(5)

In [ ]:
# виведемо числові колонки
df.describe()

In [ ]:
df[["Industry", "CompTotal", "WorkExp"]].pivot_table(index="Industry")

In [ ]:
df[["Industry", "CompTotal", "WorkExp"]].pivot_table(index="Industry", aggfunc="mean" )

In [ ]:
df[["Industry", 'Country', "CompTotal", "WorkExp"]].pivot_table(index=["Industry",'Country'], aggfunc="mean")

In [ ]:
df[["Industry", "Country", "CompTotal", "WorkExp"]].pivot_table(
    index=["Industry", "Country"], aggfunc="mean"
).to_clipboard()

In [ ]:
df[["Industry", "Country", "CompTotal", "WorkExp"]].pivot_table(
    index=["Industry", "Country"], aggfunc="median"
)

In [ ]:
df[["Industry", "Country", "CompTotal", "WorkExp"]].pivot_table(
    index=["Industry", "Country"], aggfunc="sum"
)

In [ ]:
df[["Industry", "Country", "CompTotal", "WorkExp"]].pivot_table(
    index=["Industry", "Country"], aggfunc=['mean', 'median']
)

In [ ]:
df.pivot_table(index="Industry", values="ConvertedCompYearly", aggfunc="mean").head(5)

In [ ]:
mask = df["Country"].isin(["Ukraine", "Poland", "USA", "Germany", "France", "Canada", "Australia", 'Afghanistan'])
df[mask].pivot_table(
    index="Industry", values="ConvertedCompYearly", columns="Country", aggfunc="mean", #fill_value=0
).head(5)

In [ ]:
mask = df["Country"].isin(
    [
        "Ukraine",
        "Poland",
        "USA",
        "Germany",
        "France",
        "Canada",
        "Australia",
        "Afghanistan",
    ]
)
df[mask].pivot_table(
    index="Industry",
    values="ConvertedCompYearly",
    columns="Country",
    aggfunc="mean",  
    fill_value=0,
    margins=True, # додаємо рядок зі зведеними даними
).head(5)

In [ ]:
# ключами що хочемо рахувати 
mask = df["Country"].isin(
    [
        "Ukraine",
        "Poland",
        "USA",
        "Germany",
        "France",
        "Canada",
        "Australia",
        "Afghanistan",
    ]
)
#table = 
df[mask].pivot_table(
    index="Industry",
    values=["ConvertedCompYearly", "WorkExp"],
    columns="Country",
    aggfunc={"ConvertedCompYearly": ["mean", "median"], "WorkExp": "mean"},
    fill_value=0,
).head(5)

In [ ]:
columns = ["ConvertedCompYearly", "WorkExp"]

In [ ]:
table.columns

In [ ]:
table.columns.to_flat_index() # перетворюємо в один рівень

In [ ]:
columns = ["_".join(col).strip() for col in table.columns.to_flat_index()]
columns

In [ ]:
table.columns = columns

In [ ]:
table.head(5)

In [ ]:
dfp = df.pivot_table(index="Industry", columns = 'RemoteWork', values="ConvertedCompYearly", aggfunc="mean")
sns.heatmap(dfp, annot=True, fmt=".1f", linewidths=0.5, cmap="YlGnBu")

# Lambda - функції та метод apply()

In [ ]:
dfp["Remote"]

# якщо ми використовуємо агрегацію mean, то за замовчуванням ми отримуємо не округлені числа з dtype: float64
# що ми можемо зробити інакше, якщо нам треба, наприклад, експортувати дані з 2 знаками після коми?

In [ ]:
# dfp = df.pivot_table(
#     index="Industry", columns="RemoteWork", values="ConvertedCompYearly", aggfunc=np.round(np.mean,2)
# )


# і ми не можемо використати як аргумент одразу кілька функцій

In [ ]:
def round_mean(x):
    return np.round(np.mean(x), 2)  # округлюємо до 2 знаків після коми

In [ ]:
dfp = df.pivot_table(
    index="Industry",
    columns="RemoteWork",
    values="ConvertedCompYearly",
    aggfunc=round_mean,
)

In [ ]:
dfp.head(5)

In [ ]:
lambda_round_mean = lambda x: np.round(np.mean(x), 2)  # округлюємо до 2 знаків після коми

In [ ]:
dfp = df.pivot_table(
    index="Industry",
    columns="RemoteWork",
    values="ConvertedCompYearly",
    aggfunc=lambda_round_mean,
)

# ми можемо використати lambda-функцію за спрощеним синтаксисом

In [ ]:
dfp = df.pivot_table(
    index="Industry",
    columns="RemoteWork",
    values="ConvertedCompYearly",
    aggfunc=lambda x: np.round(np.mean(x), 2),
)

dfp.head(5)

# але вона нам треба тільки один раз, тому ми можемо використати анонімну функцію вказавши прямо на місці значення відповідного аргументу

In [ ]:
# А що робити якщо ми маэмо вже створену не округлену таблицю?


In [ ]:
dfp = df.pivot_table(
    index="Industry",
    columns="RemoteWork",
    values="ConvertedCompYearly",
    aggfunc='mean',
)

dfp.head(5)

In [ ]:
dfp['Remote'].apply(lambda x: np.round(x, 2)).head(5)

In [ ]:
dfp[['Hybrid (some remote, some in-person)','In-person','Remote']].apply(lambda x: np.round(x, 2)).head(5)

# застосувати до декількох конкретиних стовпчиків

In [ ]:
dfp[["Hybrid (some remote, some in-person)", "In-person", "Remote"]].apply(lambda x: np.round(x, 2))

# Робота з датами та часом

https://github.com/devua/csv/tree/master/salaries
Зимове опитування обіцяють надати тільки в понеділок((  https://dou.ua/lenta/sitenews/winter-survey-2023/ 

In [ ]:
df = pd.read_csv('2023_june_raw.csv', sep = ';')
df.head(5)

In [ ]:
df.info()

In [ ]:
df["Timestamp"]

# поле з датою, за замовчуванням, має тип "object"

In [ ]:
#df["Timestamp"].dt.day_name()

# на поле такого типу ми не можемо застосувати методи, які доступні для типу "datetime"

In [ ]:
# ми можемо конвертувати поле в тип "datetime" за допомогою методу to_datetime

df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [ ]:
df["Timestamp"].dt.day_name()

In [ ]:
# також можна вказати поля з датами і часом при читанні файлу

df = pd.read_csv('2023_june_raw.csv', sep = ';', parse_dates=['Timestamp'])
df.info()

In [ ]:
df['Timestamp'].dt.day_name()

In [ ]:
df["Timestamp"].dt.year

In [ ]:
df["Timestamp"].dt.days_in_month

In [ ]:
# перевіримо коли відбулося перше та останне заповнення опитування

print(df["Timestamp"].min())
print(df["Timestamp"].max())

In [ ]:
df_filter = (df["Timestamp"] >= pd.to_datetime("2023-06-01")) & (df["Timestamp"] <= pd.to_datetime("2023-06-16"))
df.loc[df_filter].head(2)

# відфільтруємо лише тих, хто заповнив опитування в першій половині червня 2023 року

In [ ]:
df.index

In [ ]:
df = df.set_index("Timestamp")
df.head(5)

# Дату та час можна використовувати як індекс